选定一个起始坐标点  
确定 tile 长度 / 区域包含 tile 数目
给出区域的对角坐标点

In [1]:
import pandas as pd
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from geopy.distance import geodesic
###
# 选择一个区域
###

# 确定每个小方格的对应现实的多少米
tile_len = 1
# 确定一块区域的包含多少方格
region_resolution = (512, 512)

# 左上角坐标
left_top_coordinate = (52.3902, 9.7411)
print(f'左上角坐标: {left_top_coordinate}')

def diagonal_point(point, dist):
    """
    给出一个点和距离，求出该点所在矩形的对角点
    :param point: 点坐标
    :param dist: 距离
    :return: 对角点坐标
    """
    diagonal_len = (dist ** 2 + dist ** 2) ** 0.5
    d = geodesic(kilometers=diagonal_len).destination(point=point, bearing=135)
    return d.latitude, d.longitude


# 对角点坐标
right_bottom_coordinate = diagonal_point(left_top_coordinate, (region_resolution[0] / 1000))
print(f'右下角坐标: {right_bottom_coordinate}')

左上角坐标: (52.3902, 9.7411)
右下角坐标: (52.385598535655696, 9.748619860733815)


提取出所有在该区域内的轨迹点

In [2]:
def extract_all_trajectory_points(param_left_top_coordinate, param_right_bottom_coordinate):
    """
    传入两个对角坐标,划定一块区域
    :param param_left_top_coordinate: (latitude, longitude)
    :param param_right_bottom_coordinate: (latitude, longitude)
    """
    df = pd.read_csv(r'C:\Users\legen\Documents\001-论文\数据集\Hannover, Germany\hannover_table_with_coords.csv')
    print('Try select all trajectory point in this region...')
    # 筛选出所有在坐标区域的点
    df = df[(df['latitude'] > param_right_bottom_coordinate[0]) & (df['latitude'] < param_left_top_coordinate[0]) & (df['longitude'] > param_left_top_coordinate[1]) & (df['longitude'] < param_right_bottom_coordinate[1])]
    # 保存到文件中
    output_file = r'data\region_demo.csv'
    print(f'save to file -> [{output_file}]')
    df.to_csv(output_file, index=False)

extract_all_trajectory_points(param_left_top_coordinate=left_top_coordinate, param_right_bottom_coordinate=right_bottom_coordinate)
print('-------------- ok -----------------')

Try select all trajectory point in this region...
save to file -> [data\region_demo.csv]
-------------- ok -----------------


In [7]:
def compute_index(param_latitude, param_longitude, param_left_top_coordinate,
                  param_latitude_interval, param_longitude_interval):
    """
    传入指定坐标,根据其相对矩形区域左下角坐标(我们是将此点作为坐标原点),给出所在 tile 的索引
    :param param_latitude: 目标坐标 latitude
    :param param_longitude: 目标坐标 longitude
    :param param_left_top_coordinate: 此处必须对应为左上角的点/正好对应 vector 矩阵的索引顺序否则图片会出现问题
    :param param_latitude_interval: tile 在 latitude 的坐标间隔
    :param param_longitude_interval: tile 在 longitude 的坐标间隔
    :return: 
    """
    tile_latitude_index = int((param_left_top_coordinate[0] - param_latitude) / param_latitude_interval)
    tile_longitude_index = int((param_longitude - param_left_top_coordinate[1]) / param_longitude_interval)
    return tile_latitude_index, tile_longitude_index

   
def compute_trajectory_tile_vector(param_region_resolution, param_left_top_coordinate,
                                   param_right_bottom_coordinate, param_dataframe, save_path):
    """
    计算 1024 x 1024 向量矩阵
    :param param_region_resolution: 
    :param param_left_top_coordinate: 
    :param param_right_bottom_coordinate: 
    :param param_dataframe: 
    :param save_path: 
    :return: 
    """
    block_nums = param_region_resolution[0]
    latitude_interval = (param_left_top_coordinate[0] - param_right_bottom_coordinate[0]) / block_nums
    longitude_interval = (param_right_bottom_coordinate[1] - param_left_top_coordinate[1]) / block_nums
    print(f'tile_latitude_interval -> {latitude_interval}')
    print(f'tile_longitude_interval -> {longitude_interval}')

    tile_vector = torch.zeros(block_nums, block_nums)
    
    tile_lat_idx_list = []
    tile_lon_idx_list = []
    
    def handle_row(row):
        try:
            latitude_index, longitude_index = compute_index(row['latitude'], row['longitude'],
                                                            left_top_coordinate, latitude_interval, longitude_interval)
            # 保存下每个点所在的 tile 的 index
            tile_lat_idx_list.append(latitude_index)
            tile_lon_idx_list.append(longitude_index)
            # 计算每个 tile 中的点的个数
            tile_vector[latitude_index][longitude_index] += 1
        except Exception:
            print('##############################')
            print(f'错误数据: {row}')
            print('##############################')
        
    param_dataframe.apply(handle_row, axis=1)
    
    # 使用 pandas Series 创建新列
    tile_lat_idx = pd.Series(tile_lat_idx_list, name='tile_lat_idx')
    tile_lon_idx = pd.Series(tile_lon_idx_list, name='tile_lon_idx')
    # 将新列拼接到原 dataframe 中去
    param_dataframe = pd.concat([param_dataframe, tile_lat_idx, tile_lon_idx], axis=1)
    
    # 保存 dataframe 到文件中
    param_dataframe.to_csv(f'data/region_demo_{region_resolution}_indexed.csv', index=False)

    print('###################################')
    print(f'sum vector -> \n{tile_vector}')
    print('###################################')
    
    # 使用 sigmoid 函数将其转为 0.5 ~ 1 之间的值(没有负值)
    tile_vector = F.sigmoid(tile_vector)

    print('###################################')
    print(f'sigmoid vector -> \n{tile_vector}')
    print('###################################')
    
    # 从 -1 ~ 1 映射为 0 ~ 255
    # vector = ((tile_vector + 1) * 127.5).int()
    # 由于数据点过少 改为从 0 ~ 1 -> 64 ~ 255
    start_num = 0
    end_num = 255
    vector = (start_num + ((tile_vector - 0.5) * 2) * (end_num - start_num)).int()
    
    print('###################################')
    print(f'final vector -> \n{vector}')
    print('###################################')
        
    # 保存图片
    plt.imsave(save_path, vector, cmap='gray', vmin=0, vmax=255)

df = pd.read_csv('data/region_demo.csv', header=0)
compute_trajectory_tile_vector(region_resolution, left_top_coordinate, right_bottom_coordinate, df, 'data/region_demo.png')
print('-------------- ok -----------------')

tile_latitude_interval -> 8.987235047469566e-06
tile_longitude_interval -> 1.4687227995732743e-05
###################################
sum vector -> 
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
###################################
###################################
sigmoid vector -> 
tensor([[0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
        ...,
        [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000]])
###################################
###################################
final vector -> 
tensor([[0, 0, 0,  